In [1]:
import pandas as pd

# TODO Clean this

In [3]:
hashimoto: pd.DataFrame = pd.read_csv('../data/raw/external/Hashimoto2022_chimefrbcat1.csv')
hashimoto

,tns_name,previous_name,repeater_name,ra,ra_err,ra_notes,dec,dec_err,dec_notes,gl,...,weight_fluence_error_m,weight,weight_error_p,weight_error_m,weighted_logrhoA,weighted_logrhoA_error_p,weighted_logrhoA_error_m,weighted_logrhoB,weighted_logrhoB_error_p,weighted_logrhoB_error_m
0,FRB20180725A,180725.J0613+67,-9999,93.42,0.04,-9999,67.10,0.20,-9999,147.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FRB20180727A,180727.J1311+26,-9999,197.70,0.10,-9999,26.40,0.30,-9999,24.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FRB20180729A,180729.J1316+55,-9999,199.40,0.10,-9999,55.58,0.08,-9999,115.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FRB20180729B,180729.J0558+56,-9999,89.90,0.30,-9999,56.50,0.20,-9999,156.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FRB20180730A,180730.J0353+87,-9999,57.39,0.03,-9999,87.20,0.20,-9999,125.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,FRB20190701A,-9999,-9999,277.50,0.20,-9999,59.00,0.20,-9999,88.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
595,FRB20190701B,-9999,-9999,302.90,0.20,-9999,80.20,0.20,-9999,112.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,FRB20190701C,-9999,-9999,96.40,0.20,-9999,81.60,0.30,-9999,132.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,FRB20190701D,-9999,-9999,112.10,0.20,-9999,66.70,0.20,-9999,149.28,...,0.000649,1.361506,0.574701,0.206776,0.813142,0.485293,0.109293,0.894742,0.25519,0.19035


In [4]:
data = hashimoto.groupby('repeater_name').groups
data.pop('-9999') # Removes non repeaters
repeaters = pd.DataFrame({'name': data.keys(), 'sample_id':data.values()})
repeaters = repeaters.explode(['sample_id'])
df = hashimoto.reset_index()
rp = repeaters.merge(df[['index','tns_name']], right_on='index', left_on='sample_id')#.rename(columns={'tns_name': 'samples'})[['name', 'samples']]
rp.groupby('name').tns_name.aggregate(list)
rptes = rp.groupby('name').tns_name.aggregate(list).items()
rptrs = pd.DataFrame(rptes, columns=['name', 'samples'])
rptrs['source'] = 'Hashimoto2022'
rptrs.to_json('../data/raw/collected/repeaters.json', orient='records', indent=4)

In [80]:
# Convert FRBSTATS Repeaters list into our JSON
df = pd.read_json('../data/raw/external/FRBSTAT2022-11-23_repeaters.json')['children'].explode()
df = pd.DataFrame.from_records([*df])
children = df['children'].explode().reset_index().groupby('index').apply(lambda x: x['children'].apply(lambda x: x.get('name')))
df = df.reset_index().merge(children, on='index').rename(columns={'children_y': 'samples'})[['name', 'samples']]
df.to_json('../data/raw/collected/FRBSTATS2022-11-23_repeaters.json', orient='records', indent=4)